## Задание 1
Скачайте с сайта https://grouplens.org/datasets/movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.



In [52]:
import pandas as pd

# movies/ratings.csv - оценки пользователей
ratings = pd.read_csv('movies/ratings.csv')
movies  = pd.read_csv('movies/movies.csv').set_index('movieId')
top_movies = ratings[ratings.rating == 5.0]['movieId']

# по-умолчанию результат value_counts идёт по-убыванию
top_movieId = top_movies.value_counts().index[0]

print('Top rated movieId: {}'.format(top_movieId))
print('Most rated with 5.0 movie is: {}'.format(movies['title'][top_movieId]))

Top rated movieId: 318
Most rated with 5.0 movie is: Shawshank Redemption, The (1994)


## Задание 2
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.



In [93]:
import pandas as pd

df = pd.read_csv('power.csv')
df.head()

,country,year,quantity,category
0,Austria,1996,5.0,1
1,Austria,1995,17.0,1
2,Belgium,2014,0.0,1
3,Belgium,2013,0.0,1
4,Belgium,2012,35.0,1


In [94]:
countries  = 'latvia lithuania estonia'.split()
categories = [4, 12, 21]
years      = range(2005, 2011)

countries_rows = df.country.str.lower().isin(countries) 
categories_rows= df.category.isin(categories)
other_rows = df.year.isin(years) & df.quantity > 0.0 
qsum = df[countries_rows & categories_rows & other_rows ].quantity.sum()

print('Summary power consumption: {}'.format(qsum))

Summary power consumption: 240481.0


## Задание 3
Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
https://www.finanz.ru/valyuty/v-realnom-vremeni



In [151]:
import pandas as pd

def parse_moex_etfs(src_url='https://www.moex.com/msn/etf', skip_cols=None, index_col=None):
    """
    Спарсить таблицу etf мосбиржи. Вернёт или dataframe или кинет исключение.
    
    Parameters:
      src_url(str) : url for parsing table.
      skip_cols([str]) : columns names to skip
      
    Returns:
      pandas.DataFrame для стянутых данных.
    """
    
    def str2int(s):
        """
        Вспомогательная функция конвертации строки в целое, чтобы 
        пропуска промежуточные заголовки
        """
        try:
            return int(s)
        except:
            return None
    
    if not skip_cols:
        skip_cols = ['№', 'презентация']
        
    tbl_attrs = {"class": "table1"}
    converters= {'№': str2int}
    tables = pd.read_html(src_url, attrs=tbl_attrs, converters=converters)
    if len(tables) != 1:
        raise Exception("Expected exactly 1 table on '{}' with table attrs: {}".format(src_url, tbl_attrs))
    res = tables[0].dropna()
    res = res[[c for c in res.columns if c.lower() not in skip_cols]]
    if index_col:
        res = res.set_index(index_col)
    return res

df = parse_moex_etfs(index_col='Тикер')

In [148]:
df.head()

,Фонд,Наименование Провайдера,Базовый актив,Валюта iNAV,Формат
Тикер,,,,,
FXRB,FinEx Tradable Russian Corporate Bonds UCITS ...,FinEx Funds plc (ФинЭкс Фандс плс),Индекс российских корпоративных облигаций EMRU...,₽,ETF
FXRU,FinEx Tradable Russian Corporate Bonds UCITS ...,FinEx Funds plc (ФинЭкс Фандс плс),Индекс российских корпоративных облигаций EMRU...,$,ETF
RUSB,ITI Funds Russia-focused USD,ITI Funds (АйТиАй Фандс),ITI Funds Russia-focused USD Eurobond Index (I...,$,ETF
SBCB,Сбербанк – Индекс МосБиржи российских ликвидны...,"АО ""Сбербанк Управление Активами""",Индекс МосБиржи российских ликвидных еврооблиг...,₽/$,БПИФ
VTBB,ВТБ – Российские корпоративные облигации смарт...,АО ВТБ Капитал Управление активами (ВТБК УА),Индекс корпоративных облигаций (RUCBITR),₽,БПИФ
